# ModSSC | Transductive SSL

Run a transductive semi supervised method and evaluate it.

## Objective
- Show the minimal steps to run this component in a notebook setting.
- Provide the exact objects to look at (outputs, shapes, metrics) to confirm it worked.

## Prerequisites
- Python 3.11+.
- `pip install modssc`.
- Optional dependencies depend on datasets and backends. If an import fails, install the matching extra and rerun.

## Outline
1) Imports and configuration
2) Core run (the part that does the work)
3) Sanity checks and outputs



## Notebook notes

This notebook demonstrates the `modssc.transductive` brick.

Goals:

- build a kNN graph from features
- run a classic transductive method (label propagation)
- evaluate predictions on a toy dataset


## Imports and configuration



In [ ]:
import numpy as np

from modssc.evaluation import accuracy
from modssc.graph import GraphBuilderSpec, build_graph
from modssc.graph.artifacts import NodeDataset
from modssc.transductive import available_methods, get_method_class, get_method_info
from modssc.transductive.methods.classic.label_propagation import LabelPropagationSpec

## Synthetic data

We create three clusters with known labels.


In [ ]:
rng = np.random.default_rng(0)
n_per = 100
d = 16
X = np.vstack(
    [
        rng.normal(loc=-2.0, scale=0.8, size=(n_per, d)),
        rng.normal(loc=0.0, scale=0.8, size=(n_per, d)),
        rng.normal(loc=2.0, scale=0.8, size=(n_per, d)),
    ]
).astype(np.float32)
y_true = np.array([0] * n_per + [1] * n_per + [2] * n_per, dtype=np.int64)
X.shape, np.unique(y_true)

## Build a kNN graph

We use a small kNN graph for the demo.


In [ ]:
spec = GraphBuilderSpec(
    scheme="knn",
    metric="cosine",
    k=15,
    symmetrize="mutual",
    self_loops=True,
)
g = build_graph(X, spec=spec, seed=0, cache=False)
g.n_nodes, g.edge_index.shape

## Labeled / unlabeled split

We label a few points per class.


In [ ]:
train_mask = np.zeros(X.shape[0], dtype=bool)
for c in np.unique(y_true):
    idx = np.flatnonzero(y_true == c)
    train_mask[rng.choice(idx, size=5, replace=False)] = True

y_obs = y_true.copy()
y_obs[~train_mask] = -1

data = NodeDataset(X=X, y=y_obs, graph=g, masks={"train_mask": train_mask})
train_mask.sum(), (~train_mask).sum()

## Label propagation

Classic diffusion with hard clamping on labeled nodes.


In [ ]:
method_cls = get_method_class("label_propagation")
method = method_cls(spec=LabelPropagationSpec(max_iter=200, tol=1e-6, norm="rw"))
method.fit(data)
proba = method.predict_proba(data)
pred = proba.argmax(axis=1)

acc_all = accuracy(y_true, pred)
acc_unlabeled = accuracy(y_true[~train_mask], pred[~train_mask])
acc_all, acc_unlabeled

## Inspect available methods


In [ ]:
available_methods()

get_method_info("label_propagation")

## CLI

List and inspect methods from the command line.


In [ ]:
import subprocess
import sys


def run_cli(*args):
    cmd = [sys.executable, "-m", "modssc", *args]
    res = subprocess.run(cmd, text=True, capture_output=True)
    return res.returncode, res.stdout.strip(), res.stderr.strip()


print(run_cli("transductive", "methods", "list"))
print(run_cli("transductive", "methods", "info", "label_propagation"))

## Outputs

- The last cells should print key shapes and a minimal metric or artifact summary.
- If something fails early, the error should point to a missing optional dependency.


## Next steps
- Explore the adjacent notebooks in this folder for the other pipeline components.
- If you hit an optional dependency error, install the suggested extra and rerun.
